#### Read all the Jason files in a AWS s3 directory generated from production

In [ ]:
import pandas as pd
import numpy as np
import json
import boto3

In [7]:
BUCKET_NAME="sf-use1-training-data"
PREFIX="clm/data/"

In [8]:
# Setting up access the AWS s3
s3=boto3.client("s3")
response=s3.list_objects_v2(Bucket=BUCKET_NAME,Prefix=PREFIX)

In [9]:
# Pull the all file names in the above BUCKET/path
file_names=[obj["Key"] for obj in response.get("Contents", [])]

In [1]:
# remove the first element in the filename list becaus the file is not relevant
first_val = file_names.pop(0) 
file_names

#### Flatten each Jason file into tabular data and then concanate tabular data into one table 

In [ ]:
#Generate file list in full s3 path and filename
new_list=[]
for file in file_names:
    new="s3://"+BUCKET_NAME+"/"+file
    new_list.append(new)

In [2]:
df_init = pd.DataFrame()

# The front and end substrings to be removed from the original file folder/name string
str_remove = "s3://sf-m/lsr_data/lsr_"
str_remove2=".log"

for new in new_list:
    try:      
        df_output=pd.read_json(new, lines=True, storage_options={"anon":False})
        df_log=df_output['output'].apply(pd.Series)
        df_log=df_log['prediction'].apply(pd.Series)       
        new_string = new.replace(str_remove, "")
        new_string = new_string.replace(str_remove2, "")
        #print('new_string', new_string)
        df_log['log_date']=new_string
        df_init = pd.concat([df_init, df_log], axis=0)
    except:
        print("No output data exists at {}".format(new))
        pass
  
df_init.info()

In [3]:
df_init.head()

#### Save the data 

In [47]:
pd.to_pickle(df_init, 's3://sf-m/flatten_prediction.pkl')df_init[['clm_id','ptl_id','col_id','log_date']].to_csv('s3://sf-m/target.csv', index=False)
df_init[['clm_id','log_date']].to_csv('s3://sf-m/target.csv', index=False)